In [ ]:
# res = compare_experiments_barplot(
#     experiment_paths=[experiment_output_dir],
#     title="TARS eval.",
# )


## WANDB dev

In [ ]:
sys.path.append('../train/')

from pathlib import Path

import pandas as pd
import yaml

from data_util import create_multi_label_train_test_splits

CONFIG = yaml.safe_load(
    Path(
        "../train/train_config.yaml"
    ).read_bytes()
)

# 1.1 create splits
df = pd.read_csv(CONFIG["dataset"])
train_split, test_split = create_multi_label_train_test_splits(
    df, label_col=CONFIG["label_col"], test_size=CONFIG["test_size"]
)
test_split, dev_split = create_multi_label_train_test_splits(
    test_split, label_col=CONFIG["label_col"], test_size=CONFIG["test_size"]
)

# # 1.2 log splits
# with wandb.init(
#     project=CONFIG["wandb_project"],
#     name="reddit_aus_finance",
#     group=CONFIG["wandb_group"],
#     entity="cool_stonebreaker",
# ) as run:
#     log_dataframe(run, train, "train_split", "Train split")
#     log_dataframe(run, dev, "dev_split", "Dev split")
#     log_dataframe(run, test, "test_split", "Test split")


## Dictionary classifier

In [ ]:
from model_util import fit_and_log_dictionary_classifier, fit_and_log_linear_svc

for model in CONFIG["models"]:
    model["model"]
    # if model['name'] == 'dictionary_classifier':
    #     fit_and_log_dictionary_classifier(train, dev, test, model)

    # elif model['name'] == 'sklearn_linear_svc':
    #     fit_and_log_linear_svc(train, dev, test, model)

    # else:
    #     print(f"Unsupported model: {model['name']} found")


## Flair

In [ ]:
from flair.data import Corpus, Sentence, Token
from flair.models import SequenceTagger, TARSClassifier, TARSTagger, TextClassifier
from flair.tokenization import SegtokTokenizer

sent = Sentence("hello world", use_tokenizer=SegtokTokenizer())


In [ ]:
from eval_util import create_classification_report
from model.flair_tars import predict_flair_tars

test_preds = test_split.assign(
    pred=test_split[CONFIG["text_col"]].apply(lambda y: predict_flair_tars(y, tars))
)

classification_report = create_classification_report(test_split, test_preds, CONFIG)


In [ ]:
from data_util import label_dictionary_to_label_mat

label_dictionary_to_label_mat(test_preds.label)

label_dictionary_to_label_mat(test_preds.pred)


In [ ]:
# with wandb.init(
#         project=CONFIG["wandb_project"],
#         name=model_config["type"],
#         group=CONFIG["wandb_group"],
#         entity=CONFIG["wandb_entity"],
#     ) as run:
#     run.dir


In [ ]:
import json
import tempfile

with tempfile.TemporaryDirectory() as artefact_dir:
    (Path(artefact_dir) / "label_dictionary.json").write_text(json.dumps({"a": 10}))
    (Path(artefact_dir) / "label_dictionary.json").read_text()
    # run.save(str(Path(artefact_dir) / 'label_dictionary.json'))


In [ ]:
from model.flair_tars import fit_and_log_flair_tars_classifier

tars = fit_and_log_flair_tars_classifier(
    train_split, dev_split, test_split, CONFIG, CONFIG["models"][-1]
)


## WANDB misc

In [ ]:
import wandb

api = wandb.Api()  # refresh state of project?
_ = [
    run.delete()
    for run in api.runs(path="cool_stonebreaker/tyre_kick")
    if run.name == "inter_group_model_comparison"
]


In [ ]:
# clear out for dev purposes
import wandb

api = wandb.Api()

_ = [run.delete() for run in api.runs(path="cool_stonebreaker/tyre_kick")]


In [ ]:
[run.name for run in api.runs(path="cool_stonebreaker/tyre_kick")]
# log_inter_group_model_comparisons(project_artifacts, CONFIG)


In [ ]:
proj = api.project("blog-multi-label-train")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")
sns.set_palette("pastel", 12)

# plot results
g = sns.catplot(
    x="label",
    y="f1-score",
    hue="type",
    data=(
        group_model_classification_reports.pipe(
            lambda x: x[~x["label"].str.contains("accuracy|samples|macro|micro")]
        )
    ),
    height=10,
    kind="bar",
    ci=None,
)
g.despine(left=True)
g.set_xticklabels(rotation=45)


## Label formatting

In [ ]:
x = """workplace
boss, co-workers, WFH, life balance, office, culture, hybrid

property
refinance, real estate, property, landlord, loan, buy, house, rate, rent, resident, afford, mortgage, bedroom, townhouse, auction, agent, defect, layout, floor plan, builder, boom, salary

tax
tax, land tax, gst, salary sacrifice

insurance
insurance, indemnity, income protection

super
super, contribution, fund, balance, self-funded, retire, pension

public institution
watch dog, rba, central bank, mint, fair work, bond

inflation
inflation, interest rates, reserve bank, phillip lowe, rba, petrol

exchange
exchange, rate, dollar

stocks
stock, shares, invest, indexed, van guard, wealth, assets, asx, commsec, etf, return, vdhg, high growth, selfwealth, dividends, securities, buy, dip, 200

toxic
butt, salty, fuck, laughable, fool, tard, lol, bro, shit"""

In [ ]:
label_dicts = {}
for e in x.split('\n\n'):
    label_dicts[e.split('\n')[0]] = sorted(e.split('\n')[1].split(', '))

In [ ]:
{'workplace': ['WFH', 'boss', 'co-workers', 'culture', 'hybrid', 'life balance', 'office'],
'property': ['afford', 'agent', 'auction', 'bedroom', 'boom', 'builder', 'buy', 'defect', 'floor plan', 'house', 'landlord', 'layout', 'loan', 'mortgage', 'property', 'rate', 'real estate', 'refinance', 'rent', 'resident', 'salary', 'townhouse'],
'tax': ['gst', 'land tax', 'salary sacrifice', 'tax'],
'insurance': ['income protection', 'indemnity', 'insurance'],
'super': ['balance', 'contribution', 'fund', 'pension', 'retire', 'self-funded', 'super'],
'public institution': ['bond', 'central bank', 'fair work', 'mint', 'rba', 'watch dog'],
'inflation': ['inflation', 'interest rates', 'petrol', 'phillip lowe', 'rba', 'reserve bank'],
'exchange': ['dollar', 'exchange', 'rate'],
'stocks': ['200', 'assets', 'asx', 'buy', 'commsec', 'dip', 'dividends', 'etf', 'high growth', 'indexed', 'invest', 'return', 'securities', 'selfwealth', 'shares', 'stock', 'van guard', 'vdhg', 'wealth'],
'toxic': ['bro', 'butt', 'fool', 'fuck', 'laughable', 'lol', 'salty', 'shit', 'tard']}

## Save novel plot

In [52]:
sys.path.append("../train/")
from eval_util import *
import wandb


def log_inter_group_model_comparisons(project_artifacts, CONFIG):
    group_model_classification_reports = []
    # format, concat
    for idx, record in (
        project_artifacts.query('_sequence_name == "test_classification_report"')
    ).iterrows():
        group_model_classification_reports.append(
            (
                parse_wandb_table_artifact(record.artifact)
                .assign(type=record.type)
                .assign(group=record.group)
            )
        )
    group_model_classification_reports = pd.concat(group_model_classification_reports)

    # choose single most performant model from each group
    group_model_classification_reports = (
        group_model_classification_reports
        .groupby("group")
        .apply(get_most_performant_classifier_per_group)
        .reset_index(drop=True)
    )

    # create plot
    fig = px.bar(
        (
            group_model_classification_reports.pipe(
                lambda x: x[~x["label"].str.contains("accuracy|samples|macro|micro")]
            )
        ),
        x="label",
        y="f1-score",
        color="type",
        barmode="group",
    )
    return fig

    # # log plot
    # with wandb.init(
    #     project=CONFIG["wandb_project"],
    #     name="inter_group_model_comparison",
    #     group="inter_group_model_comparison",
    #     entity=CONFIG["wandb_entity"],
    #     job_type="aux_plot",
    # ) as run:
    #     run.log({"inter_group_model_comparison": fig})

api = wandb.Api()
project_artifacts = list_all_project_artifacts(api, CONFIG)

# log_inter_group_model_comparisons(project_artifacts, CONFIG)

In [62]:
wandb.restore

<function wandb.sdk.wandb_run.restore(name: str, run_path: Union[str, NoneType] = None, replace: bool = False, root: Union[str, NoneType] = None) -> Union[NoneType, TextIO]>

In [65]:
run_records

,run,type,group,_sequence_name,artifact
2,<Run cool_stonebreaker/tyre_kick/axlfxb47 (fin...,flair_tars,annotation_1,test_preds,<Artifact QXJ0aWZhY3Q6MjAyNzY3ODAw>
4,<Run cool_stonebreaker/tyre_kick/3n7um7a3 (fin...,sklearn_linear_svc,annotation_1,test_preds,<Artifact QXJ0aWZhY3Q6MjAyNzY3NTk3>
6,<Run cool_stonebreaker/tyre_kick/tg2ec19u (fin...,dictionary_classifier,annotation_1,test_preds,<Artifact QXJ0aWZhY3Q6MjAyNzY3NDQy>


In [66]:
run_records

,run,type,group,_sequence_name,artifact
2,<Run cool_stonebreaker/tyre_kick/axlfxb47 (fin...,flair_tars,annotation_1,test_preds,<Artifact QXJ0aWZhY3Q6MjAyNzY3ODAw>
4,<Run cool_stonebreaker/tyre_kick/3n7um7a3 (fin...,sklearn_linear_svc,annotation_1,test_preds,<Artifact QXJ0aWZhY3Q6MjAyNzY3NTk3>
6,<Run cool_stonebreaker/tyre_kick/tg2ec19u (fin...,dictionary_classifier,annotation_1,test_preds,<Artifact QXJ0aWZhY3Q6MjAyNzY3NDQy>


## Model restoration

In [55]:
# flair
from flair.models import TARSClassifier

project_artifacts.iloc[0]
wandb.restore('best-model.pt', '/'.join(project_artifacts.iloc[0].run.path))

tars = TARSClassifier.load('./best-model.pt')

run               <Run cool_stonebreaker/tyre_kick/axlfxb47 (fin...
type                                                     flair_tars
group                                                  annotation_1
_sequence_name                                           test_preds
artifact                        <Artifact QXJ0aWZhY3Q6MjAyNzY3ODAw>
Name: 2, dtype: object

<_io.TextIOWrapper name='/root/blog-multi-label/notebooks/best-model.pt' mode='r' encoding='UTF-8'>

2022-09-26 00:07:10,087 loading file ./best-model.pt


In [57]:
from flair.data import Corpus, Sentence
from flair.tokenization import SegtokTokenizer

def create_flair_classification_sentence(text, label_object, label_type="class"):
    sentence = Sentence(text, use_tokenizer=SegtokTokenizer())
    for label in [k for k, v in label_object.items() if v > 0]:
        sentence.add_label(label_type, label, 1.0)
    return sentence


def predict_flair_tars(text, flair_tars_model):
    sentence = Sentence(text)
    labels = flair_tars_model.get_current_label_dictionary().get_items()
    flair_tars_model.predict(sentence)
    pred_dict = {label: 0.0 for label in labels}
    for e in sentence.labels:
        label = e.to_dict()["value"]
        confidence = round(float(e.to_dict()["confidence"]), 2)
        pred_dict[label] = confidence
    return pred_dict

predict_flair_tars('hello world', tars)

{'public_institution': 0.0,
 'property': 0.0,
 'stocks': 0.0,
 'tax': 0.0,
 'exchange': 0.0,
 'inflation': 0.0,
 'toxic': 0.0,
 'super': 0.0,
 'insurance': 0.0,
 'workplace': 0.0}

In [ ]:
# sklearn
from joblib import load

wandb.restore('model.joblib', '/'.join(project_artifacts.query('type == "sklearn_linear_svc"').iloc[0].run.path))
sklearn_linear_svc = load('./model.joblib')

In [ ]:
def predict_sklearn_linear_svc(text, sklearn_linear_svc_model):
    return dict(
        zip(
            label_dictionary_to_label_mat(
                test_split[CONFIG["label_col"]]
            ).columns.values,
            sklearn_linear_svc_model.predict([text])[0].toarray()[0],
        )
    )

In [ ]:
# dictionary classifier
wandb.restore('label_dictionary.json', '/'.join(project_artifacts.query('type == "dictionary_classifier"').iloc[0].run.path))

from clear_bow.classifier import DictionaryClassifier
import srsly

dc = DictionaryClassifier(classifier_type='multi_label', label_dictionary=srsly.read_json('./label_dictionary.json'))
dc.predict_single('hello world')